Created on Lundi 11 January 2021

Group 5 - Classification
Extraction features syntaxe

@authors : F.B

In [1]:
import pandas as pd 
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline

from sklearn.metrics import f1_score
from string import punctuation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_analyse = pd.read_json("df_final_clean.json")
df_analyse.drop('art_content', axis=1, inplace=True)

df_train = pd.read_csv("Data_With_Features_Syntax.csv")
df_train = df_train.merge(df_analyse)
df_train.index = df_train.art_id.values

In [5]:
def clean_keyword(link_keyword : str) -> list:
  ''' This function allows you to clean up keywords
    Input :
      link_keyword : link of keyword
    Output : 
      list_key_word : list of cleaned keywords 
  '''
  df_lexique: pd.DataFrame = pd.read_csv(link_keyword, sep="  ", header=None)
  df_lexique.columns: list = ['key_word']
  list_key_word = df_lexique.key_word.values.tolist()
  list_key_word_net : list = []

  for k in list_key_word :
    k=k.lower()
    k=k.replace(' de ',' ')
    k=k.replace(" d' ",' ')
    k=k.replace("."," ")
    k=k.replace("-"," ")
    k=k.replace(" du "," ")
    list_key_word_net.append(k)

  list_key_word_net = pd.unique(list_key_word_net).tolist()
  return list_key_word_net

link_keyword = "/content/drive/MyDrive/G5 Inter-Promo 2021/Ressources/Lexique_Innovation.txt"
list_key_word = clean_keyword(link_keyword)

In [6]:
# Counts the number of words
def nb_word(text: str) -> int:
    """Documentation
      Parameters:
        text: text of the article

      Out (if exists):
        nb_word: number of word in  the text
    """
    nb_words: list = []
    nb: int = 0
    # removes special characters
    for p in punctuation:
      text= text.replace(p, ' ')

    # counts the number of words present in the text
    return len(text.split())

In [7]:
# Count the number of time where the words in the list appear

def count_key_words(text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        t : number of key word in the text
    """
    t : int = 0

    if text != None:
        text = text.lower()
        text = text.split()
        for j in range(len(text)):
            if (text[j] in list_key_word):   #list_key_word: List of word that we will check in the sentences
                t += 1
            else : # For 2-word keywords
              try : # In case j+1 does not exist
                if text[j]+' '+text[j+1] in list_key_word :
                  t += 1
                else : #For 3-word keywords
                  try : # In case j+2 does not exist
                    if text[j]+' '+text[j+1]+' '+text[j+2] in list_key_word :
                      t+=1
                  except :
                    pass
              except :
                pass
    return t

In [12]:
# Count the number of sentence
def phrases( text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        n:  The number of sentence in a article
    """

    n : int = 0
   
    if not isinstance(text, str):
        text: str = str(text)

    if (text != None):
        text = text.replace("..", ".")
        text = text.replace("...", ".")
        text = text.replace("!", ".")
        text = text.replace("!!", ".")
        text = text.replace("!!!", ".")
        text = text.replace("?", ".")
        text = text.replace("??", ".")
        text = text.replace("???", ".")
        text = text.replace("?!", ".")
        text = text.replace("!?", ".")
        n = len(sent_tokenize(text))

    return n

In [8]:
def create_features(data : pd.DataFrame, text : str, title : str) -> pd.DataFrame:
  ''' This function allows you to create features
    Input :
      data : Dataframe
      text : Article contents column
      title : Article title column
    Output :
      data : Dataframe
  '''
  data["Nb_key_words"] : np.DataFrame = data[text].apply(count_key_words)
  data["Nb_key_words_title"] : np.DataFrame = data[title].apply( count_key_words)
  data["Nb_words"]: np.DataFrame = data[text].apply(nb_word)
  data["Nb_words_title"]: np.DataFrame = data[title].apply(nb_word)
  data["Nb_sentences"]: np.DataFrame = data['art_content'].apply(phrases)
  data["average_word_sentence"]: np.DataFrame = data["Nb_words"] / data["Nb_sentences"]
  data['ratio_key_words']: np.DataFrame  = data['Nb_key_words']/data['Nb_words']
  data['ratio_key_words']: np.DataFrame  = data['ratio_key_words'].fillna(0)
  data['ratio_key_sentences']: np.DataFrame  = data['Nb_key_words']/data['Nb_sentences']
  data['ratio_key_sentences']: np.DataFrame  = data['ratio_key_sentences'].fillna(0)
  data['ratio_key_word_title']: np.DataFrame  = data['Nb_key_words_title'] / data['Nb_words_title']
  data['ratio_key_word_title']: np.DataFrame  = data['ratio_key_word_title'].fillna(0)
  
  return data

In [14]:
def slope_intercept(A : list, B : list) -> tuple:
  '''This function is used to calculate the steering coefficient and 
     the y-intercept at the origin of a straight line passing through 
     the 2 points passed as a parameter.
     Input : 
      A : point 1
      B : point 2
     Output :
        a : slope
        b : intercept
  '''
  a = ( B[1] - A[1] ) / ( B[0] - A[0] )
  b = A[1] - a * A[0]
  return a,b

def classification_group(C : list, A: list, B :list) -> float:
  ''' This function calculates the difference between the Y-axis of 
      the point and its Y-axis corresponds to the line
      Input :
        A,B,C : points
      Output :
  '''
  a,b = slope_intercept(A, B)
  return C[1]-a*C[0]-b

In [17]:
def create_label( data : pd.DataFrame, text : str, title : str ):
  ''' This function is used to create the innovation label. 
      The details of the study are contained in the notebook g5_FB_Innovation_approach_pca_V1_1.ipynb
    Input :
      data : Dataframe
      text : Article contents column
      title : Article title column
    Output :
      data : Dataframe
  '''

  #
  data = create_features(df_train, text, title)
  #
  data_label = data [['Nb_key_words', 'Nb_key_words_title', 'ratio_key_words', 'ratio_key_word_title','ratio_key_sentences']]
  data_label.fillna('', inplace=True)

  #
  data_scale = scaler.fit_transform(data_label)
  pca = PCA(n_components=2)
  Xp = pca.fit_transform(data_scale)

  #
  index_group_bottom : list = []
  gb : list = []
  index_group_top : list = []
  gh : list = []
  art_id : list = data.index.values.tolist()

  # Value to be automated later
  A1 :list = [9.8, 19] 
  B1 :list = [9.8, -15]
  A2 :list = [-5, 5]
  B2 :list = [ 20, -15]

  for i in tqdm(range(Xp.shape[0])) :
    if Xp[i,0]< A1[0] : #Excluding outliers
      # Classification between the 2 groups
      if classification_group(Xp[i,:], A2, B2) >0 :
        index_group_top.append(art_id[i])
        gh.append(i)
      else :
        index_group_bottom.append(art_id[i])
        gb.append(i)   

  # Labelling
  data_scale = pd.DataFrame(data_scale, columns=data_label.columns, index = data.index)
  data_scale['label_innovation'] = -1 # unknown
  data_scale.loc[index_group_bottom,'label_innovation'] = 0 # No innovation
  data_scale.loc[index_group_top,'label_innovation'] = 1 # Innovation

  # Predict the unknown
  train = data_scale[data_scale['label_innovation']!=-1]
  X = train[['Nb_key_words','Nb_key_words_title','ratio_key_words','ratio_key_word_title','ratio_key_sentences']].values
  y = train.label_innovation.values 

  test = data_scale [data_scale['label_innovation']==-1]
  X_test = test[['Nb_key_words','Nb_key_words_title','ratio_key_words','ratio_key_word_title','ratio_key_sentences']].values

  # Regression logistic
  model = LogisticRegression()
  model.fit(X,y)
  y_pred= model.predict(X_test)
  df_pred = pd.DataFrame( y_pred, columns=['label_innovation'], index = test.index)

  #
  label=train[['label_innovation']]
  label=label.append(df_pred)
  label.sort_index(inplace=True)

  data.sort_index(inplace=True)
  data['label_innovation']=label.label_innovation.tolist()
  return data

In [16]:
%%time
df_train = create_label(df_train, 'art_content_prepd', 'art_title')


CPU times: user 29.9 s, sys: 106 ms, total: 30.1 s
Wall time: 30.1 s


In [ ]:
#df_train.to_json('Data_With_Features_Syntax_v1.json')